# Prices

In [1]:
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

### CPI components 


https://www.bls.gov/cpi/tables/relative-importance/home.htm

In [2]:
# Series stored as a dictionary
series = {'CUUR0000SA0': 'All items',
          'CUUR0000SAM': 'Medical care',
          'CUUR0000SAH': 'Housing',
          'CUUR0000SEFV': 'Food away from home',
          'CUUR0000SAF11': 'Food at home',
          'CUUR0000SAR': 'Recreation',
          'CUUR0000SAE1': 'Education',
          'CUUR0000SAT': 'Transportation',
          'CUUR0000SAA': 'Apparel',
          'CUUR0000SA0E': 'Energy',
          'CUUR0000SAE2': 'Communication'}

url = 'https://www.bls.gov/cpi/tables/relative-importance/2019.txt'
rel_imp = pd.read_csv(url, header=None, delim_whitespace=True)
r = requests.get(url)
data = r.text

wgts = {value: re.findall(value+r'\..*\s+(\d+\.\d+)\s+\d+\.\d+\s+\r', data)[0] 
        for key, value in series.items()}

In [3]:
# Start year and end year
years = (2018, 2020)
df = bls_api(series, years, bls_key)

Post Request Status: REQUEST_SUCCEEDED


In [4]:
wgt_date = pd.to_datetime('2019-12-01')
dates = {'Latest': df.index[-1], 'Previous': df.index[-13]}

d = {}

for name, date in dates.items():

    updated_wgts = {srs: ((df.loc[date, srs] / df.loc[wgt_date, srs])
                     * float(wgts[srs]) / 
                     (df.loc[date, 'All items'] / 
                      df.loc[wgt_date, 'All items']))
                    for key, srs in series.items()}
    
    d[name] = {srs: (df[srs].pct_change(12).loc[date] 
                        * (updated_wgts[srs]))
                  for key, srs in series.items()}
    
results = pd.DataFrame(d).drop('All items').round(2).sort_values('Latest', ascending=False)
results.to_csv(data_dir / 'cpi_comp.csv', index_label='name')

mo1  = df.index[-1].strftime('%b %Y')
write_txt(text_dir / 'cpi_mo1.txt', mo1)

mo2  = df.index[-13].strftime('%b %Y')
write_txt(text_dir / 'cpi_mo2.txt', mo2)

results

,Latest,Previous
Housing,0.85,1.27
Medical care,0.45,0.22
Food at home,0.36,0.05
Food away from home,0.21,0.19
Communication,0.09,-0.05
Education,0.07,0.09
Recreation,0.06,0.04
Apparel,-0.17,-0.02
Energy,-0.69,-0.14
Transportation,-0.89,-0.05


In [5]:
ltdate = df.index[-1].strftime('%B %Y')
prevdate = df.index[-13].strftime('%B %Y')

ltd = {i: (results.iloc[i].name, results['Latest'].iloc[i]) for i in [0, 1, -2, -1]}

textd = {}
for idx, (name, val) in ltd.items():
    if idx == 0:
        name = name.lower()
    if val >= 0.05:
        textd[idx] = f'{name} contributed {val:.2f} percentage points to'
    elif val <= -0.04:
        textd[idx] = f'{name} subtracted {abs(val):.2f} percentage points from'
    else:
        textd[idx] = f'{name} did not contribute significantly to'
        
prevd = {i: (results.iloc[i].name, results['Previous'].iloc[i]) for i in [0, 1, -2, -1]}

textd2 = {}
for idx, (name, val) in prevd.items():
    if val >= 0.05:
        textd2[idx] = f'a contribution of {val:.2f} percentage points'
    elif val <= -0.04:
        textd2[idx] = f'a reduction of {abs(val):.2f} percentage points'
    else:
        textd2[idx] = 'virtually no effect on inflation'

In [6]:
text = (f'In {ltdate}, {textd[0]} overall CPI inflation, compared to '+
        f'{textd2[0]} in {prevdate}. {textd[1]} overall inflation in '+
        f'{ltdate}, compared to {textd2[1]} in {prevdate}.')

write_txt(text_dir / 'cpicomp.txt', text)

print(text)

text2 = (f'{textd[-1]} overall CPI inflation in {ltdate}, compared to '+
         f'{textd2[-1]} in {prevdate}. {textd[-2]} overall inflation, '+
         f'compared to {textd2[-2]} the previous year.')

write_txt(text_dir / 'cpicomp2.txt', text2)

text2

In July 2020, housing contributed 0.85 percentage points to overall CPI inflation, compared to a contribution of 1.27 percentage points in July 2019. Medical care contributed 0.45 percentage points to overall inflation in July 2020, compared to a contribution of 0.22 percentage points in July 2019.


'Transportation subtracted 0.89 percentage points from overall CPI inflation in July 2020, compared to a reduction of 0.05 percentage points in July 2019. Energy subtracted 0.69 percentage points from overall inflation, compared to a reduction of 0.14 percentage points the previous year.'

### CPI Decomposition

In [7]:
# Relative weights for series of interest, from here: 
# https://www.bls.gov/cpi/tables/relative-importance/home.htm
rel_wgt = {'CUUR0000SAF1': [(('2009-12-01', '2011-12-01'), 13.738),
                           (('2011-12-01', '2013-12-01'), 14.308),
                           (('2013-12-01', '2015-12-01'), 13.891), 
                           (('2015-12-01', '2017-12-01'), 14.015), 
                           (('2017-12-01', '2019-12-01'), 13.384),
                           (('2019-12-01', '2021-12-01'), 13.771)],
           'CUUR0000SA0': [(('2009-12-01', '2011-12-01'), 100.0),
                           (('2011-12-01', '2013-12-01'), 100.0),
                           (('2013-12-01', '2015-12-01'), 100.0), 
                           (('2015-12-01', '2017-12-01'), 100.0), 
                           (('2017-12-01', '2019-12-01'), 100.0),
                           (('2019-12-01', '2021-12-01'), 100.0)],
           'CUUR0000SA0E': [(('2009-12-01', '2011-12-01'), 8.553),
                            (('2011-12-01', '2013-12-01'), 9.679),
                            (('2013-12-01', '2015-12-01'), 9.046), 
                            (('2015-12-01', '2017-12-01'), 6.816), 
                            (('2017-12-01', '2019-12-01'), 7.513),
                            (('2019-12-01', '2021-12-01'), 6.706)],
           'CUUR0000SAH1': [(('2009-12-01', '2011-12-01'), 32.289),
                            (('2011-12-01', '2013-12-01'), 31.539),
                            (('2013-12-01', '2015-12-01'), 32.029), 
                            (('2015-12-01', '2017-12-01'), 33.15), 
                            (('2017-12-01', '2019-12-01'), 32.843),
                            (('2019-12-01', '2021-12-01'), 33.158)],
           'CUUR0000SACL1E': [(('2009-12-01', '2011-12-01'), 21.276),
                              (('2011-12-01', '2013-12-01'), 19.852),
                              (('2013-12-01', '2015-12-01'), 19.71), 
                              (('2015-12-01', '2017-12-01'), 19.613), 
                              (('2017-12-01', '2019-12-01'), 19.849),
                              (('2019-12-01', '2021-12-01'), 20.137)],
           'CUUR0000SASLE': [(('2009-12-01', '2011-12-01'), 56.432),
                             (('2011-12-01', '2013-12-01'), 56.161),
                             (('2013-12-01', '2015-12-01'), 57.353), 
                             (('2015-12-01', '2017-12-01'), 59.556), 
                             (('2017-12-01', '2019-12-01'), 59.254),
                             (('2019-12-01', '2021-12-01'), 59.387)]}


series = {key: key for key, value in rel_wgt.items()}

# Start year and end year
years = (2008, 2020)
df = bls_api(series, years, bls_key)

# Dictionary combining all the info for each series
d = {i: {'name': i,
         'values': df[i],
         'rel_wgt': rel_wgt[i]} for i in list(rel_wgt.keys())}

# Adjust for changes to relative importance
df1, df2, df3, df4, df5, df6 = pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
for i, v in d.items():
    start, end = v['rel_wgt'][0][0][0], v['rel_wgt'][0][0][1]
    rwc, rwn = v['rel_wgt'][0][1], v['rel_wgt'][1][1]
    df1.at[start: end, i] = (v['values'].loc[start: end])
    df1[i] = (df1[i].diff().cumsum() / df1.loc[start, i] + 1)
    df1.at[start, i] = 1.0
    df1[i] = (df1[i] * rwc)
    link = (df1.loc[end, i] / rwn)
    
    # Next set of dates
    start, end = v['rel_wgt'][1][0][0], v['rel_wgt'][1][0][1]
    rwc, rwn = v['rel_wgt'][1][1], v['rel_wgt'][2][1]
    df2[i] = (v['values'].loc[start: end])
    df2[i] = df2[i].diff().cumsum() / df2.loc[start, i] + 1
    df2.at[start, i] = 1.0
    df2[i] = (df2[i] * rwc) * link
    link = (df2.loc[end, i] / rwn)
    
    # Next set of dates
    start, end = v['rel_wgt'][2][0][0], v['rel_wgt'][2][0][1]
    rwc, rwn = v['rel_wgt'][2][1], v['rel_wgt'][3][1]
    df3[i] = (v['values'].loc[start: end])
    df3[i] = df3[i].diff().cumsum() / df3.loc[start, i] + 1
    df3.at[start, i] = 1.0
    df3[i] = (df3[i] * rwc) * link
    link = (df3.loc[end, i] / rwn)
    
    # Next set of dates
    start, end = v['rel_wgt'][3][0][0], v['rel_wgt'][3][0][1]
    rwc, rwn = v['rel_wgt'][3][1], v['rel_wgt'][4][1]
    df4[i] = (v['values'].loc[start: end])
    df4[i] = df4[i].diff().cumsum() / df4.loc[start, i] + 1
    df4.at[start, i] = 1.0
    df4[i] = (df4[i] * rwc) * link
    link = (df4.loc[end, i] / rwn)

    # Next set of dates
    start, end = v['rel_wgt'][4][0][0], v['rel_wgt'][4][0][1]
    rwc, rwn = v['rel_wgt'][4][1], v['rel_wgt'][5][1]
    df5[i] = (v['values'].loc[start: end])
    df5[i] = df5[i].diff().cumsum() / df5.loc[start, i] + 1
    df5.at[start, i] = 1.0
    df5[i] = (df5[i] * rwc) * link
    link = (df5.loc[end, i] / rwn)    
    
    # Next set of dates
    start, end = v['rel_wgt'][5][0][0], v['rel_wgt'][5][0][1]
    rwc = v['rel_wgt'][5][1]
    df6[i] = (v['values'].loc[start: end])
    df6[i] = df6[i].diff().cumsum() / df6.loc[start, i] + 1
    df6.at[start, i] = 1.0
    df6[i] = (df6[i] * rwc) * link
    
# Calculate exactly what I want to show
final = pd.concat([df1, df2, df3, df4, df5, df6])  # Combine the various relative importance dfs
final = final[~final.index.duplicated(keep='first')].diff(12) # Drop duplicate pivot year data
# Core services is services less food, energy, and shelter
final['core_services'] = final['CUUR0000SASLE'] - final['CUUR0000SAH1']
# Combine food and energy
final['food_energy'] = final['CUUR0000SAF1'] + final['CUUR0000SA0E']
final = final.dropna().round(2)
d2 = final[['CUUR0000SACL1E', 'core_services', 'CUUR0000SAH1', 'food_energy']].loc['2011-01-01':]
# Rename columns
col_names = ['core_goods', 'core_services', 'shelter', 'food_energy']
d2.columns = col_names
d2['total'] = final['CUUR0000SA0'].loc['2011-01-01':]

d2.to_csv(data_dir / 'cpi_decomp.csv', index_label='date')

ltdate = dtxt(d2.index[-1])['mon1']
prdate = dtxt(d2.index[-13])['mon1']
cg = cont_subt(d2.core_goods.iloc[-1])
cs = cont_subt(d2.core_services.iloc[-1], style='end')
sh = cont_subt(d2.shelter.iloc[-1], style='end')
fe = cont_subt(d2.food_energy.iloc[-1], style='end')
tot = d2.total.iloc[-1]
cgpr = cont_subt(d2.core_goods.iloc[-13], style='end')
cspr = cont_subt(d2.core_services.iloc[-13], style='end')
shpr = cont_subt(d2.shelter.iloc[-13], style='end')
fepr = cont_subt(d2.food_energy.iloc[-13], style='end')
totpr = d2.total.iloc[-13]
text = (f'As of {ltdate}, core goods {cg} the overall non-seasonally-'+
        f'adjusted CPI inflation rate of {tot:.2f} percent, '+
        f'while core services excluding shelter {cs}. Shelter {sh}, '+
        f'and food and energy {fe}. One year prior, in {prdate}, '+
        f'the corresponding CPI inflation rate was {totpr:.2f} percent; '
        f'core goods {cgpr}, core services excluding shelter {cspr}, '+
        f'shelter {shpr}, and food and energy {fepr}.')
write_txt(text_dir / 'cpi_decomp.txt', text)
print(text)

Post Request Status: REQUEST_SUCCEEDED
As of July 2020, core goods subtracted 0.11 percentage point from the overall non-seasonally-adjusted CPI inflation rate of 1.17 percent, while core services excluding shelter contributed 0.65 percentage point. Shelter contributed 0.97 percentage point, and food and energy subtracted 0.38 percentage point. One year prior, in July 2019, the corresponding CPI inflation rate was 2.11 percent; core goods contributed 0.09 percentage point, core services excluding shelter contributed 0.56 percentage point, shelter contributed 1.40 percentage points, and food and energy contributed 0.09 percentage point.


### CPI chart

In [8]:
# Series stored as a dictionary
series = {
    'CUSR0000SA0': 'ALL', 
    'CUSR0000SA0L1E': 'CORE'}

# Start year and end year
years = (1988, 2020)
df = bls_api(series, years, bls_key)

(df.pct_change(12).dropna() * 100).to_csv(data_dir / 'cpi.csv', index_label='date')

Post Request Status: REQUEST_SUCCEEDED


### CPI text

In [9]:
s = pd.read_csv(data_dir / 'cpi.csv')
date = pd.to_datetime(s['date'].iloc[-1]).strftime('%B %Y')
core = s['CORE'].iloc[-1]
allitems = s['ALL'].iloc[-1]

text = ('\href{https://www.bls.gov/cpi/}{Consumer prices} increased by '+
        f'{allitems:.1f} percent over the year ending {date}, according '+
        'to the CPI for all urban consumers. Core inflation, which does '+
        'not include the more volatile food and energy prices, was '+
        f'{core:.1f} percent.')

write_txt(text_dir / 'cpi_main.txt', text)

text

'\\href{https://www.bls.gov/cpi/}{Consumer prices} increased by 1.0 percent over the year ending July 2020, according to the CPI for all urban consumers. Core inflation, which does not include the more volatile food and energy prices, was 1.6 percent.'

### PPI

In [2]:
ppi = (fred_df('PPIACO', start='1988')['VALUE'].pct_change(12) * 100).dropna()
ppi.to_csv(data_dir / 'ppi.csv', index_label='date')
node = end_node(ppi, 'green!80!blue', date=True)
write_txt(text_dir / 'ppi_node.txt', node)

ch = inc_dec_percent(ppi.iloc[-1])
prval = ppi.iloc[-13]
compare = compare_text(ppi.iloc[-1], prval, [1.0, 3.0, 5.0])
date = dtxt(ppi.index[-1])['mon1']
date2 = dtxt(ppi.index[-13])['mon1']

text = ('The Bureau of Labor Statistics \\href{https://www.bls.gov/ppi/}{reports} '+
        'prices producers receive for the various goods and services they produce. '+
        'The producer price index for all commodities (see {\color{green!80!blue}\\textbf{---}}) '+
        f'{ch} over the year ending {date}, {compare} '+
        f'the 12-month growth rate of {prval:.1f} percent '+
        f'in {date2}.')

write_txt(text_dir / 'ppi_main.txt', text)
print(text)

The Bureau of Labor Statistics \href{https://www.bls.gov/ppi/}{reports} prices producers receive for the various goods and services they produce. The producer price index for all commodities (see {\color{green!80!blue}\textbf{---}}) decreased by 3.9 percent over the year ending July 2020, slightly below the 12-month growth rate of -1.8 percent in July 2019.


### Import/Export Price Index

In [2]:
# Series stored as a dictionary
series = {'EIUIR': 'Imports', 
          'EIUIQ': 'Exports',
          'EIUIREXFUELS': 'ImpExFuels',
          'EIUIR10': 'ImpFuels',
          'EIUIQEXAG': 'ExpExAg',
          'EIUIQAG': 'ExpAg'}

# Start year and end year
years = (1988, 2020)
df = bls_api(series, years, bls_key)

df.to_csv(data_dir / 'mxpi_main.csv', index_label='date')

srs = ['Imports', 'Exports']
(df[srs].pct_change(12).dropna() * 100).to_csv(data_dir / 'mxpi.csv', index_label='date')

Post Request Status: REQUEST_SUCCEEDED


In [3]:
df = pd.read_csv(data_dir / 'mxpi_main.csv', index_col='date')

df.index = pd.to_datetime(df.index)

data = (df.pct_change(12).dropna() * 100)

ltval = data['Imports'].iloc[-1]
ch = inc_dec_percent(ltval)
ltdate = dtxt(data.index[-1])['mon1']
chpr = inc_dec_percent(data['Imports'].iloc[-2])
prdate = dtxt(data.index[-2])['mon1']

s = series_info(data['Imports'])
m3y = s['three_year_mean']
lm = f", {s['last_matched']}" if s['days_since_match'] > 1000 else ''
compare = compare_text(ltval, m3y, [0.4, 2.0, 6.0])

text = ('According to the Bureau of Labor Statistics, US import '+
        'prices (see {\color{cyan!85!yellow}\\textbf{---}}) '+
        f'{ch} over the 12 months ending {ltdate}{lm}. '+
        f'Import prices {chpr} over the 12 months ending the month prior ({prdate}). '
        f'The latest value is {compare} the past-three-year average growth rate '+
        f'of {m3y:.1f} percent.')

write_txt(text_dir / 'mxpi_imp.txt', text)

text

'According to the Bureau of Labor Statistics, US import prices (see {\\color{cyan!85!yellow}\\textbf{---}}) decreased by 3.8 percent over the 12 months ending June 2020. Import prices decreased by 6.2 percent over the 12 months ending the month prior (May 2020). The latest value is substantially below the past-three-year average growth rate of 0.4 percent.'

In [4]:
ltval = data['Exports'].iloc[-1]
ch = inc_dec_percent(ltval)
ltdate = dtxt(data.index[-1])['mon1']
chpr = inc_dec_percent(data['Exports'].iloc[-2])
prdate = dtxt(data.index[-2])['mon1']

s = series_info(data['Exports'])
m3y = s['three_year_mean']
lm = f", {s['last_matched']}, and " if s['days_since_match'] > 1000 else ', '
compare2 = compare_text(ltval, m3y, [0.4, 2.0, 6.0])
also = 'also ' if compare2 == compare else ''

text = ('Prices of US exports (see {\color{red!25!orange}\\textbf{---}}) '+
        f'{ch} over the year ending {ltdate}{lm}{also}{compare2} '+
        f'its three-year moving average growth rate of {m3y:.1f} percent. '+
        f'Export prices {chpr} over the 12 months ending {prdate}.')

write_txt(text_dir / 'mxpi_exp.txt', text)

text

'Prices of US exports (see {\\color{red!25!orange}\\textbf{---}}) decreased by 4.4 percent over the year ending June 2020, also substantially below its three-year moving average growth rate of 0.6 percent. Export prices decreased by 6.2 percent over the 12 months ending May 2020.'

### PCE Price Index

In [2]:
data = pd.DataFrame()

df = fred_df('PCEPI', start='1988').pct_change(12)
data['PCE'] = df['VALUE'].dropna() * 100.0
df = fred_df('PCEPILFE', start='1988').pct_change(12)
data['CORE'] = df['VALUE'].dropna() * 100.0

data.to_csv(data_dir / 'pce_pi.csv', index_label='date')

ltdate = dtxt(data.index[-1])['mon1']
prdate = dtxt(data.index[-2])['mon1']
pryrdate = dtxt(data.index[-13])['mon1']
ltval = data.PCE.iloc[-1]
prval = data.PCE.iloc[-2]
pryrval = data.PCE.iloc[-13]
ltcore = data.CORE.iloc[-1]
prcore = data.CORE.iloc[-2]
pryrcore = data.CORE.iloc[-13]

text = (f'As of {ltdate}, PCE inflation, measured as the one year percent '+
        f'change in the overall index, is {ltval:.1f} percent, compared to '+
        f'{prval:.1f} percent in {prdate}, and {pryrval:.1f} percent in {pryrdate}. '+
        f'Core PCE inflation, excluding food and energy, was {ltcore:.1f} percent '+
        f'in {ltdate}, {prcore:.1f} percent in {prdate}, and {pryrcore:.1f} '+
        f'percent in {pryrdate}.')

write_txt(text_dir / 'pce_inf_basic.txt', text)

print(text)

As of June 2020, PCE inflation, measured as the one year percent change in the overall index, is 0.8 percent, compared to 0.5 percent in May 2020, and 1.5 percent in June 2019. Core PCE inflation, excluding food and energy, was 0.9 percent in June 2020, 1.0 percent in May 2020, and 1.7 percent in June 2019.
